# Principais Funções para Testes de Hipóteses

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from scipy import stats as st

### Trazendo os Dados

In [8]:
dados_ab = pd.read_csv(r"..\13-Teste-Hipoteses\Dados\exampleDataABtest.csv")
dados_ab.head()

,group,time,clickedTrue
0,A,2016-06-02 02:17:53,0
1,A,2016-06-02 03:03:54,0
2,A,2016-06-02 03:18:56,1
3,B,2016-06-02 03:23:43,0
4,A,2016-06-02 04:04:00,0


In [10]:
dados_nps = pd.read_csv(r"..\13-Teste-Hipoteses\Dados\nps_example.csv", sep=";")
dados_nps.head()

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male


# Teste t

- Comparar os Grupos

In [15]:
dados_ab.groupby('group') \
    .agg(media_cliques = pd.NamedAgg('clickedTrue', 'mean'),
         dp_cliques = pd.NamedAgg('clickedTrue', 'std'),
         n = pd.NamedAgg('clickedTrue', 'count'))

,media_cliques,dp_cliques,n
group,,,
A,0.04,0.196155,500
B,0.08,0.271565,500
